In [1]:
from lusidtools.jupyter_tools import toggle_code

"""Access Management in LUSID

Attributes
----------
IAM
Access Management
Policies
"""

toggle_code("Toggle Docstring")

In [114]:
from lusid.utilities import ApiClientFactory
import lusid as lu
import pandas as pd
from pprint import pprint
import json

api_factory = ApiClientFactory()

# Confirm success by printing SDK version
api_status = pd.DataFrame(api_factory.build(lu.ApplicationMetadataApi).get_lusid_versions().to_dict())
display(api_status)

,api_version,build_version,excel_version,links
0,v0,0.6.10456.0,0.5.3126,"{'relation': 'RequestLogs', 'href': 'http://fb..."


In [115]:
import finbourne_identity as identity
import finbourne_identity.rest

configuration = identity.Configuration(
    host = f'{api_factory.api_client.configuration.host[:-4]}/identity'
)
configuration.access_token = api_factory.api_client.configuration.access_token

identity_client = identity.ApiClient(configuration)

In [116]:
import finbourne_access as access
configuration = access.Configuration(
    host = f'{api_factory.api_client.configuration.host[:-4]}/access'
)

configuration.access_token = api_factory.api_client.configuration.access_token
access_client = access.ApiClient(configuration)

In [117]:
from datetime import datetime
from datetime import timedelta
today = f'{datetime.today().isoformat()}+00:00'
tomorrow = datetime.today() + timedelta(days = 1)
tomorrow = f'{tomorrow.isoformat()}+00:00'

In [118]:
# don't show exception if error is due to upsert
def exception_guard(e, code):
    return e.status and e.status != '400 Bad Request' and e.body and json.loads(e.body)["code"] == code

set up an example user.

In [119]:
api_instance = identity.UsersApi(identity_client)
create_user_request = {"firstName":"Joe","lastName":"Bloggs","emailAddress":"joe.bloggs@myco.com","login":"joe.bloggs@myco.com","type":"Personal"} # CreateUserRequest | Details of the User to be created
wait_for_reindex = True # bool | Should the request wait until the newly created User is indexed (available in List) before returning (optional) (default to False)
try:
    api_response = api_instance.create_user(create_user_request, wait_for_reindex=wait_for_reindex)
    pprint(api_response)
except identity.rest.ApiException as e:
    if not exception_guard(e, 658):
        print("Exception when calling UsersApi->create_user: %s\n" % e)

# Access Management

In this part of the course, we will introduce aspects of access management in LUSID, and demonstrate the process of:

- Creating roles
- Writing simple policies
- Creating policy collections
- Assinging roles to users
- Building more complex policies
- Querying policy logs using Luminesce

## CREATING ROLES

A role models a real-world job function or responsibility within LUSID.


For example:  

A data controller might require write access to all the data in LUSID.  
A portfolio manager might require write access to certain portfolios. 
A risk manager might require read-only access to all portfolios.  

Each role has one or more policies, each of which grants (or denies) access to a particular feature or dataset. You can combine policies in any way you like to precisely model the professional duties of a role, and edit policies as these professional duties evolve over time.

You assign a role to one or more LUSID users. A user with that role inherits all the access rights granted by its policies. A user can have multiple roles. 

You must create a role using both the Identity API and the Access API. We have different APIs for identity management and access control to securely separate these concerns; roles are the link between the two systems.

Let's create a role!

In [120]:
api_instance = identity.RolesApi(identity_client)
create_role_request = {"name":"ResearchAnalyst","description":"Associates which require trading data"} # CreateRoleRequest | Details of the role to be created

try:
    # [EARLY ACCESS] CreateRole: Create Role
    api_response = api_instance.create_role(create_role_request)
    pprint(api_response)
except finbourne_identity.rest.ApiException as e:
    if not exception_guard(e, 157):
        print("Exception when calling RolesApi->create_role: %s\n" % e)

In [121]:
api_instance = access.RolesApi(access_client)
role_creation_request = {"code":"ResearchAnalyst","description":"Associates which require trading data", "resource":{
    "policyIdRoleResource":{"policies":[]}
    },"when":{"activate":today,"deactivate":tomorrow}}


try:
    # [EARLY ACCESS] CreateRole: Create Role
    api_response = api_instance.create_role(role_creation_request)
    pprint(api_response)
except access.rest.ApiException as e:
    if not exception_guard(e, 615):
        print("Exception when calling RolesApi->create_role: %s\n" % e)

We created a new ResearchAnalyst role.

## Writing simple policies

Even once authenticated, a user cannot access data or perform operations within LUSID until all access control checks have been performed.

A policy is a grant or denial of access to either a particular entity dataset or to a particular feature.

Upon each access request, LUSID checks each policy in every role for the calling user:
- First, LUSID checks a user’s feature policies. A feature policy allows (or denies) access to one or more API endpoints, for example to the ListPortfolios endpoint.
- Next, LUSID checks a user’s data policies. A data policy allows (or denies) access to one or more entity datasets, for example to portfolio data.
- Finally, for each data policy, LUSID checks whether a user is permitted to access properties decorated onto entities that support properties. 

Let's write a simple policy.

In [122]:
api_instance = access.PoliciesApi(access_client)
policy_creation_request = {
    "code":"Instruments-iam-example",
    "applications": [
        "LUSID"
    ],
    "grant": "Allow",
    "selectors": [
        {
            "idSelectorDefinition": {
                "identifier": {
                    "code": "api-instruments-listinstruments",
                    "scope": "LUSID"
                },
                "actions": [
                    {
                        "scope": "LUSID",
                        "activity": "Execute",
                        "entity": "Feature"
                    }
                ],
                "name": "Run ListInstruments",
                "description": "Run the ListInstruments API endpoint"
            }
        }
    ], 
    "when": {
        "activate": today,
        "deactivate": tomorrow
    }
} # PolicyCreationRequest | The definition of the Policy

try:
    # [EARLY ACCESS] CreatePolicy: Create Policy
    api_response = api_instance.create_policy(policy_creation_request)
    pprint(api_response)
except access.rest.ApiException as e:
    if not exception_guard(e, 613):
        print("Exception when calling PoliciesApi->create_policy: %s\n" % e)

We've created a simple policy with this request. This policy grants a user access to run the List Instruments endpoint within LUSID.

## Creating policy collections

A policy can be grouped in a policy collection for logical convenience, and policy collections may themselves contain policy collections. You can create a policy collection to group logically similar policies together.

In [123]:
policy_collection_creation_request = {"code":"example-policy-collection",
                                      "policies":[
                                          {"scope":"default",
                                           "code":"Instruments-iam-example"}]
                                     }
                                      # PolicyCollectionCreationRequest | The definition of the PolicyCollection

try:
    # [EARLY ACCESS]  CreatePolicyCollection: Create PolicyCollection
    api_response = api_instance.create_policy_collection(policy_collection_creation_request)
    pprint(api_response)
except access.rest.ApiException as e: 
    if not exception_guard(e, 612):
        print("Exception when calling PoliciesApi->create_policy_collection: %s\n" % e)

We've created a new policy collection containing the policy we created earlier.

## Assinging roles to users

You can assign a user many roles, and a role many policies. 

We've created a research analyst role. Now let's assign the role to a user!

In [ ]:
# Create an instance of the API class
api_instance = identity.RolesApi(identity_client)
try:
    # [EARLY ACCESS] ListRoles: List Roles
    api_response = api_instance.list_roles()
    matching_role_filter = filter(lambda role: role.name == 'ResearchAnalyst', api_response)
    matching_role = next(matching_role_filter)
    role_id = matching_role.id
except ApiException as e:
    print("Exception when calling RolesApi->list_roles: %s\n" % e)

In [ ]:
# Create an instance of the API class
api_instance = identity.UsersApi(identity_client)
include_roles = False # bool | Flag indicating that the users roles should be included in the response (optional) (default to False)
include_deactivated = False # bool | Include previously deleted (not purged) users (optional) (default to False)

try:
    # [EARLY ACCESS] ListUsers: List Users
    api_response = api_instance.list_users(include_roles=include_roles, include_deactivated=include_deactivated)
    matching_user_filter = filter(lambda role: role.email_address == "joe.bloggs@myco.com", api_response)
    matching_user = next(matching_user_filter)
    user_id = matching_user.id
except ApiException as e:
    print("Exception when calling UsersApi->list_users: %s\n" % e)

In [124]:
api_instance = identity.RolesApi(identity_client)

_id = role_id # str | The unique identifier for the Role
user_id = user_id # str | The unique identifier for the User
try:
    # [EARLY ACCESS] AddUserToRole: Add User to Role
    api_instance.add_user_to_role(_id, user_id)
except ApiException as e:
    print("Exception when calling RolesApi->add_user_to_role: %s\n" % e)

We've assigned our user the role we created earlier!

## Building more complex policies

Let's create a more complicated policy. We'll build up the policy using the FINBOURNE Access SDK, and take a look at the corresponding JSON.

Let's take a look at the create_policy method to see what's needed to build this request in Python

In [125]:
api_instance = access.PoliciesApi(access_client)
help(api_instance.create_policy)

Help on method create_policy in module finbourne_access.api.policies_api:

create_policy(policy_creation_request, **kwargs) method of finbourne_access.api.policies_api.PoliciesApi instance
    [EARLY ACCESS] CreatePolicy: Create Policy  # noqa: E501
    
    Creates a Policy  # noqa: E501
    This method makes a synchronous HTTP request by default. To make an
    asynchronous HTTP request, please pass async_req=True
    
    >>> thread = api.create_policy(policy_creation_request, async_req=True)
    >>> result = thread.get()
    
    :param policy_creation_request: The definition of the Policy (required)
    :type policy_creation_request: PolicyCreationRequest
    :param async_req: Whether to execute the request asynchronously.
    :type async_req: bool, optional
    :param _preload_content: if False, the urllib3.HTTPResponse object will
                             be returned without reading/decoding response
                             data. Default is True.
    :type _preload_cont

We need a `PolicyCreationRequest` object for this function. Lets build one.

In [126]:
help(access.PolicyCreationRequest)

Help on class PolicyCreationRequest in module finbourne_access.models.policy_creation_request:

class PolicyCreationRequest(builtins.object)
 |  PolicyCreationRequest(code=None, description=None, applications=None, grant=None, selectors=None, _for=None, _if=None, when=None, how=None, local_vars_configuration=None)
 |  
 |  NOTE: This class is auto generated by OpenAPI Generator.
 |  Ref: https://openapi-generator.tech
 |  
 |  Do not edit the class manually.
 |  
 |  Methods defined here:
 |  
 |  __eq__(self, other)
 |      Returns true if both objects are equal
 |  
 |  __init__(self, code=None, description=None, applications=None, grant=None, selectors=None, _for=None, _if=None, when=None, how=None, local_vars_configuration=None)
 |      PolicyCreationRequest - a model defined in OpenAPI"
 |      
 |      :param code:  Code of the policy being created (required)
 |      :type code: str
 |      :param description:  Description of what the policy will be used for
 |      :type descrip

We can see we need a code, description and grant as a string. We also need a `SelectorDefinition` and a `WhenSpec`. We can optionally add an `IfExpression`, `ForSpec` and a `HowSpec`.

Lets get started building this request.

In [127]:
code = "complex-policy-example"
description = "A policy which..."
applications = ['LUSID']
grant = "Allow"

We've created some of the less complex variables required in a `PolicyCreationRequest`.

Now let's build the `IdSelectorDefinition`, a more complex type.

In [128]:
help(access.IdSelectorDefinition)

Help on class IdSelectorDefinition in module finbourne_access.models.id_selector_definition:

class IdSelectorDefinition(builtins.object)
 |  IdSelectorDefinition(identifier=None, actions=None, name=None, description=None, local_vars_configuration=None)
 |  
 |  NOTE: This class is auto generated by OpenAPI Generator.
 |  Ref: https://openapi-generator.tech
 |  
 |  Do not edit the class manually.
 |  
 |  Methods defined here:
 |  
 |  __eq__(self, other)
 |      Returns true if both objects are equal
 |  
 |  __init__(self, identifier=None, actions=None, name=None, description=None, local_vars_configuration=None)
 |      IdSelectorDefinition - a model defined in OpenAPI"
 |      
 |      :param identifier:  (required)
 |      :type identifier: dict(str, str)
 |      :param actions:  (required)
 |      :type actions: list[finbourne_access.ActionId]
 |      :param name: 
 |      :type name: str
 |      :param description: 
 |      :type description: str
 |  
 |  __ne__(self, other)
 | 

We'll use this definition to build an `IdSelectorDefinition`.

In [129]:
import json
identifier = {
    "code": "api-instruments-listinstruments",
    "scope": "LUSID"
}
action_id = access.ActionId(scope= "LUSID", activity = "Execute", entity= "Feature")

execute_list_instruments_selector = access.IdSelectorDefinition(identifier = identifier, actions= [action_id])
execute_list_instruments_selector_definition = access.SelectorDefinition(id_selector_definition=execute_list_instruments_selector)

print(json.dumps(execute_list_instruments_selector_definition.to_dict(), indent = 2))

{
  "metadata_selector_definition": null,
  "id_selector_definition": {
    "identifier": {
      "code": "api-instruments-listinstruments",
      "scope": "LUSID"
    },
    "actions": [
      {
        "scope": "LUSID",
        "activity": "Execute",
        "entity": "Feature"
      }
    ],
    "name": null,
    "description": null
  },
  "match_all_selector_definition": null,
  "policy_selector_definition": null
}


Great! We've built our definition, which selects the `ListInstruments` endpoint and the json output looks similar to the previous example!

We'll create a selector so we can grant access to read instrument data.

In [130]:
identifier = {
    "scope": "*"
}

action_id = access.ActionId(scope= "default", activity = "Read", entity= "Instrument")

instrument_read_data_selector = access.IdSelectorDefinition(identifier = identifier, actions= [action_id])

instrument_read_data_selector_definition = access.SelectorDefinition(id_selector_definition=instrument_read_data_selector)


print(json.dumps(instrument_read_data_selector_definition.to_dict(), indent = 2))

{
  "metadata_selector_definition": null,
  "id_selector_definition": {
    "identifier": {
      "scope": "*"
    },
    "actions": [
      {
        "scope": "default",
        "activity": "Read",
        "entity": "Instrument"
      }
    ],
    "name": null,
    "description": null
  },
  "match_all_selector_definition": null,
  "policy_selector_definition": null
}


We've now created a selector to grant access to instrument data.

Let's now create a `WhenSpec` and a `ForSpec`.

In [131]:
when = access.WhenSpec(activate= today,deactivate = tomorrow)
effective_date_relative = access.EffectiveDateRelative(
    date = 'Now',
    adjustment=-7,
    unit='Day',
    relative_to_date_time='BeforeOrOn'
)
_for = access.ForSpec(effective_date_relative = effective_date_relative)

The `WhenSpec` allows us to specify an activation and deactivation date for the Policy. 

`ForSpec` allows you to specify a rolling validity date for a data policy. This means the policy only takes effect a set date before or after a relative point in time, such as ‘now’ or the ‘first business day of the month’.

You can use this feature to restrict access to the latest data, or conversely to only allow access to the latest data. We've decided not to let our ResearchAnalyst role to not see any Instruments until 7 days after they've been created.

We can now assemble our `PolicyCreationRequest`.

In [132]:


policy_creation_request = access.PolicyCreationRequest(code = code,
                                                       description=description,
                                                       applications=applications, 
                                                       grant=grant,
                                                       selectors=[instrument_read_data_selector_definition, execute_list_instruments_selector_definition],
                                                       _for = [_for],
                                                       when = when
                                                      )
print(json.dumps(policy_creation_request.to_dict(), indent = 2))


{
  "code": "complex-policy-example",
  "description": "A policy which...",
  "applications": [
    "LUSID"
  ],
  "grant": "Allow",
  "selectors": [
    {
      "metadata_selector_definition": null,
      "id_selector_definition": {
        "identifier": {
          "scope": "*"
        },
        "actions": [
          {
            "scope": "default",
            "activity": "Read",
            "entity": "Instrument"
          }
        ],
        "name": null,
        "description": null
      },
      "match_all_selector_definition": null,
      "policy_selector_definition": null
    },
    {
      "metadata_selector_definition": null,
      "id_selector_definition": {
        "identifier": {
          "code": "api-instruments-listinstruments",
          "scope": "LUSID"
        },
        "actions": [
          {
            "scope": "LUSID",
            "activity": "Execute",
            "entity": "Feature"
          }
        ],
        "name": null,
        "description": null

We've built our `PolicyCreationRequest` in Python, and we can see this creates a JSON similar to before.
This Policy allows Read access to instruments older than 7 days.
Let's now upload this policy to LUSID.

In [133]:
try:
    # [EARLY ACCESS] CreatePolicy: Create Policy
    api_response = api_instance.create_policy(policy_creation_request)
    pprint(api_response)
except access.rest.ApiException as e:
    if not exception_guard(e, 613):
        print("Exception when calling PoliciesApi->create_policy: %s\n" % e)

We've built a more complex policy in Python.

## Querying policy logs using Luminesce

We can use Luminesce to query access logs, seeing what users have accessed, whether they were successful or whether they did not have the correct entitlements to the LUSID feature.

In [134]:
%%luminesce 
SELECT ^ FROM Lusid.Logs.Metrics.Entitlement LIMIT 100;

,Client,User,Action,Application,Detail,ResourceId,Result
0,fbn-uni,00uj5ppmtezKb0DPC2p7,Feature/LUSID/Execute,lydia,NaN,scope:Lydia|code:api-users-createuser,Success
1,fbn-uni,00uj5ppmtezKb0DPC2p7,Feature/LUSID/Execute,lydia,NaN,scope:Lydia|code:api-users-createuser,Success
2,fbn-uni,00uj5ppmtezKb0DPC2p7,User/default/Add,lydia,NaN,login:947d083ac6df49b4a55f7afb625af2a2,Success
3,fbn-uni,00uj5ppmtezKb0DPC2p7,Feature/LUSID/Execute,lydia,NaN,scope:Lydia|code:api-roles-listroles,Success
4,fbn-uni,00uj5ppmtezKb0DPC2p7,Role/default/List,lydia,NaN,scope:default|code:ResearchAnalyst,Success
...,...,...,...,...,...,...,...
95,fbn-uni,00uihwl5ljacZQhq92p7,Feature/LUSID/Execute,shrine,NaN,licensedfeature:system-licence-check-notificat...,Success
96,fbn-uni,00uihwl5ljacZQhq92p7,Feature/LUSID/Execute,shrine,NaN,licensedfeature:system-licence-check-horizonst...,DoesNotHaveRequiredLicence
97,fbn-uni,00uihwl5ljacZQhq92p7,Feature/LUSID/Execute,shrine,NaN,licensedfeature:system-licence-check-horizonco...,DoesNotHaveRequiredLicence
98,fbn-uni,00uihwl5ljacZQhq92p7,Feature/LUSID/Execute,shrine,NaN,licensedfeature:system-licence-check-horizonsix,DoesNotHaveRequiredLicence


Here we see a list of recent access requests, providing the User id, the ActionId of the policy that was applied, the id of the resource requested, and the outcome of the access request.